In [1]:
    import hopsworks
    import pandas as pd
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    import seaborn as sns
    from matplotlib import pyplot
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import joblib
    import os
    
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error

/Users/deepakshankar/opt/anaconda3/envs/ML_DL/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/210070
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
wine_fg = fs.get_feature_group(name="wine", version=1)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="wine",
                                  version=1,
                                  description="Read from Wine quality dataset",
                                  labels=["quality"],
                                  query=query)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/210070/fs/209989/fv/wine/version/1


In [4]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

# # Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
# model = KNeighborsClassifier(n_neighbors=2)
# model.fit(X_train, y_train.values.ravel())


# Initialize the RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using Hive (5.03s) 


RandomForestRegressor(random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier


# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
# X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

# # Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
# model = KNeighborsClassifier(n_neighbors=2)
# model.fit(X_train, y_train.values.ravel())


# Initialize the RandomForestRegressor
rfc_model = RandomForestClassifier(n_estimators=200, random_state=52)

# Fit the model to the training data
rfc_model.fit(X_train, y_train.values.ravel())

RandomForestClassifier(n_estimators=200, random_state=52)

In [6]:
#For Random forest regressor
import numpy as np
from numpy import round

# Make predictions on the test set
y_rf_pred = np.round(rf_model.predict(X_test))

# Evaluate the model. Not to be used for Classification.
mse = mean_squared_error(y_test, y_rf_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 0.7445864661867806


In [8]:
# Make predictions on the test set, for Random forest classifier
y_rfc_pred = np.round(rfc_model.predict(X_test))

# # Compare predictions (y_rfc_pred) with the labels in the test set (y_test)
metrics = classification_report(y_test, y_rfc_pred, output_dict=True, zero_division=1)

In [15]:
# # Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
model = KNeighborsClassifier(n_neighbors=10)
model.fit(X_train, y_train.values.ravel())

y_knn_pred = np.round(model.predict(X_test))

metrics = classification_report(y_test, y_knn_pred, output_dict=True, zero_division=1)

print(metrics)

{'3': {'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, '4': {'precision': 0.125, 'recall': 0.047619047619047616, 'f1-score': 0.06896551724137931, 'support': 42}, '5': {'precision': 0.4462616822429907, 'recall': 0.5190217391304348, 'f1-score': 0.4798994974874372, 'support': 368}, '6': {'precision': 0.4490909090909091, 'recall': 0.5416666666666666, 'f1-score': 0.49105367793240556, 'support': 456}, '7': {'precision': 0.30985915492957744, 'recall': 0.13496932515337423, 'f1-score': 0.18803418803418803, 'support': 163}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 32}, '9': {'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'accuracy': 0.4333958724202627, 'macro avg': {'precision': 0.4757445351804968, 'recall': 0.17761096836707477, 'f1-score': 0.1754218400993443, 'support': 1066}, 'weighted avg': {'precision': 0.40315834508855186, 'recall': 0.4333958724202627, 'f1-score': 0.4071947622760454, 'support': 1066}}


In [9]:
X_train

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol
1,1,7.4,0.28,0.25,11.9,0.053,25.0,148.0,0.99760,3.10,0.62,9.2
2,1,7.5,0.24,0.31,13.1,0.050,26.0,180.0,0.99884,3.05,0.53,9.1
4,1,8.3,0.14,0.45,1.5,0.039,18.0,98.0,0.99215,3.02,0.56,11.0
5,1,6.4,0.15,0.36,1.8,0.034,43.0,150.0,0.99220,3.42,0.69,11.0
6,1,7.1,0.12,0.30,3.1,0.018,15.0,37.0,0.99004,3.02,0.52,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...
5323,1,7.9,0.16,0.30,4.8,0.037,37.0,171.0,0.99670,3.47,0.44,9.0
5324,1,7.6,0.19,0.37,13.1,0.033,52.0,151.0,0.99726,3.18,0.79,10.4
5325,1,6.7,0.24,0.37,11.3,0.043,64.0,173.0,0.99632,3.08,0.53,9.9
5326,1,6.0,0.27,0.19,1.7,0.020,24.0,110.0,0.98980,3.32,0.47,12.6


In [10]:
y_train

,quality
1,5
2,6
4,6
5,8
6,7
...,...
5323,4
5324,6
5325,6
5326,7


In [11]:
X_test

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol
0,1,6.900000,0.290000,0.320000,5.800000,0.040000,16.000000,112.000000,0.99300,3.04000,0.58000,11.20000
3,1,6.700000,0.220000,0.330000,1.200000,0.036000,36.000000,86.000000,0.99058,3.10000,0.76000,11.40000
7,0,8.200000,1.000000,0.090000,2.300000,0.065000,7.000000,37.000000,0.99685,3.32000,0.55000,9.00000
10,1,6.300000,0.360000,0.300000,4.800000,0.049000,14.000000,85.000000,0.99320,3.28000,0.39000,10.60000
20,1,6.900000,0.190000,0.490000,6.600000,0.036000,49.000000,172.000000,0.99320,3.20000,0.27000,11.50000
...,...,...,...,...,...,...,...,...,...,...,...,...
5302,1,5.800000,0.300000,0.120000,1.600000,0.036000,57.000000,163.000000,0.99239,3.38000,0.59000,10.50000
5315,1,8.900000,0.270000,0.280000,0.800000,0.024000,29.000000,128.000000,0.98984,3.01000,0.35000,12.40000
5316,1,6.600000,0.350000,0.290000,14.400000,0.044000,54.000000,177.000000,0.99910,3.17000,0.58000,8.90000
5328,1,6.900000,0.190000,0.350000,5.000000,0.067000,32.000000,150.000000,0.99500,3.36000,0.48000,9.80000


In [13]:
for i in y_rfc_pred:
    print(i)

6
6
5
6
6
6
5
7
5
7
6
5
6
6
6
6
5
6
6
5
6
5
6
5
6
5
5
5
5
5
7
5
5
6
5
6
5
6
6
7
6
5
5
6
6
6
6
7
5
6
6
5
7
6
5
5
6
6
6
6
6
5
6
5
6
6
6
6
6
6
6
5
6
6
6
6
6
5
5
5
5
6
5
6
6
6
6
5
6
5
6
5
6
6
6
6
5
6
6
5
5
6
6
5
7
6
5
6
5
5
6
5
5
6
5
5
5
5
5
5
7
6
6
5
6
6
6
6
5
6
5
8
6
6
5
7
5
7
5
6
6
5
5
5
6
5
6
6
5
5
6
6
5
6
5
6
5
5
5
6
6
5
6
6
6
6
6
6
5
6
5
5
5
5
6
6
6
6
5
7
7
6
7
4
5
6
6
6
5
5
6
5
6
5
6
5
6
6
5
5
5
6
5
6
6
5
6
6
5
6
5
6
6
6
5
6
5
5
6
6
6
5
6
5
6
6
6
6
5
5
6
5
6
6
5
6
7
6
6
6
5
6
5
7
6
6
6
7
5
6
6
6
6
6
6
4
5
6
6
5
6
5
6
5
5
6
6
6
5
5
6
5
6
5
6
6
6
5
6
6
5
6
6
6
6
6
6
6
7
5
6
5
5
5
6
6
5
5
7
7
5
7
5
7
6
5
5
6
6
7
5
6
6
6
6
6
5
5
6
6
6
6
6
6
5
6
6
5
5
7
6
6
7
7
6
6
6
4
6
5
5
6
6
6
5
6
5
6
6
6
5
5
6
6
7
5
6
6
7
6
5
5
6
6
6
4
6
5
5
5
6
6
6
6
6
5
5
7
6
7
6
6
6
5
5
6
6
6
6
6
6
6
6
6
6
6
5
7
6
6
5
5
6
6
6
5
6
5
6
6
6
6
6
5
6
5
6
6
7
6
6
6
6
6
6
6
6
6
5
6
6
6
5
6
6
5
7
5
6
6
6
6
6
5
6
6
6
7
6
6
6
7
5
5
6
5
3
6
6
6
6
5
6
6
5
5
6
5
7
5
6
6
6
6
6
5
6
6
5
7
6
5
6
5
6
6
6
6
6
7
6
7
5
6
5
5
5
6
6
5


In [14]:
print(metrics)
# metrics = classification_report(y_test, y_pred, output_dict=True)

{'3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, '4': {'precision': 0.375, 'recall': 0.07142857142857142, 'f1-score': 0.12, 'support': 42}, '5': {'precision': 0.6324786324786325, 'recall': 0.6032608695652174, 'f1-score': 0.6175243393602226, 'support': 368}, '6': {'precision': 0.551948051948052, 'recall': 0.7456140350877193, 'f1-score': 0.6343283582089553, 'support': 456}, '7': {'precision': 0.5681818181818182, 'recall': 0.3067484662576687, 'f1-score': 0.39840637450199207, 'support': 163}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 32}, '9': {'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'accuracy': 0.5769230769230769, 'macro avg': {'precision': 0.44680121465835754, 'recall': 0.24672170604845384, 'f1-score': 0.25289415315302427, 'support': 1066}, 'weighted avg': {'precision': 0.557039479178316, 'recall': 0.5769230769230769, 'f1-score': 0.5501716015681709, 'support': 1066}}


In [16]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# # Assuming X_train, X_test, y_train, y_test are your feature and target variables
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Linear Regression
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train.values.ravel())
y_pred_linear = linear_reg.predict(X_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))
print("Linear Regression RMSE:", rmse_linear)

# Support Vector Regression (SVR)
svr = SVR()
svr.fit(X_train, y_train.values.ravel())
y_pred_svr = svr.predict(X_test)
rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))
print("SVR RMSE:", rmse_svr)

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train.values.ravel())
y_pred_gbr = gbr.predict(X_test)
rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))
print("Gradient Boosting Regressor RMSE:", rmse_gbr)


Linear Regression RMSE: 0.721659066438534
SVR RMSE: 0.8114404713393576
Gradient Boosting Regressor RMSE: 0.6949564642541116


In [17]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'wine_model' directory will be saved to the model registry. Create the dir, first.
model_dir="wine_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save our rf_model(random forest) to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(rf_model, model_dir + "/wine_model.pkl")   

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
wine_model = mr.python.create_model(
    name="wine_model", 
    metrics={"rmse" : rmse},
    model_schema=model_schema,
    description="Wine Quality Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
wine_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


Uploading model files (0 dirs, 0 files):  17%|▋   | 1/6 [00:00<00:02,  2.09it/s]
Uploading: 0.000%|                        | 0/11752481 elapsed<00:00 remaining<?
Uploading: 8.922%|█▏            | 1048576/11752481 elapsed<00:02 remaining<00:26
Uploading: 26.767%|███▍         | 3145728/11752481 elapsed<00:03 remaining<00:07
Uploading: 26.767%|███▍         | 3145728/11752481 elapsed<00:03 remaining<00:07
Uploading: 35.689%|████▋        | 4194304/11752481 elapsed<00:05 remaining<00:09
Uploading: 53.533%|██████▉      | 6291456/11752481 elapsed<00:06 remaining<00:04
Uploading: 62.455%|████████     | 7340032/11752481 elapsed<00:15 remaining<00:12
Uploading: 71.377%|█████████▎   | 8388608/11752481 elapsed<00:16 remaining<00:07
Uploading: 80.300%|██████████▍  | 9437184/11752481 elapsed<00:16 remaining<00:03
Uploading: 91.078%|██████████▉ | 10703905/11752481 elapsed<00:18 remaining<00:01
Uploading: 100.000%|███████████| 11752481/11752481 elapsed<00:20 remaining<00:00
Uploading input_example and 

Model created, explore it at https://c.app.hopsworks.ai:443/p/210070/models/wine_model/1


Model(name: 'wine_model', version: 1)